In [1]:
import random
import copy
from datetime import datetime

class Team:
  def __init__(self, name, wins, games, points):
    self._name = name
    self._points = points
    self._wins = wins
    self._games = games
    if(wins != 0 and games != 0):
        self._win_rate = wins/games
    else:
        self._win_rate = 0
        
  def update_win_rate(self):
    self._win_rate = self._wins/self._games    
    
  def win(self):   
    self._games = self._games + 1
    self._wins = self._wins + 1
    self.update_win_rate()  
    
  def lose(self):
    self._games = self._games + 1 
    self.update_win_rate() 
    
  def tie(self):
    self._games = self._games + 1 
    self.update_win_rate()    
    
  def win_points(self):
    self._points = self._points + 3
    self._games = self._games + 1
    self._wins = self._wins + 1
    self.update_win_rate()  
    
  def lose_points(self):
    self._games = self._games + 1 
    self.update_win_rate() 
    
  def tie_points(self):
    self._points = self._points + 1
    self._games = self._games + 1 
    self.update_win_rate()    
    
  def __repr__(self):
    return (self._name +" : " + str(self._points) + " / "+ str(self._wins)+ " / "+ str(self._games)+ " / "+ str(self._win_rate) + "\n")


def search_team_one(gameString,teamList,team_one):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team._name == teams[0]):      
            return team

def search_team_two(gameString,teamList,team_two):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team._name == teams[1]):      
            return team

In [2]:
def simulate_game(team_one,team_two,current_team_list):
  status = "UNDEFINED"
  rng = random.uniform(0,1)
  rng_two = random.uniform(0,1)

  if(team_one._win_rate >= rng):
    if(team_two._win_rate < rng_two):
      team_one.win_points()
      team_two.lose_points()
      status = "ONE WON"   
    else:
      team_one.tie_points()
      team_two.tie_points()
      status = "TIE"
  else:
    if(team_two._win_rate >= rng_two):
      team_two.win_points()
      team_one.lose_points()
      status = "TWO WON"   
    else:
      team_one.tie_points()
      team_two.tie_points()
      status = "TIE"

def simulate_game_elm(team_one,team_two,match_history):
  status = "UNDEFINED"
  while(status == "UNDEFINED" or status == "TIE"):
        rng = random.uniform(0,1)
        rng_two = random.uniform(0,1)  
        
        if(team_one._win_rate >= rng):
            if(team_two._win_rate < rng_two):      
              team_one.win()  
              status = "WIN"
              if(team_one._name  == "Brasil" or team_two._name == "Brasil"):
                match_history.append(team_one._name+" X "+ team_two._name)
              return team_one
            else:     
              status = "TIE"
        else:
            if(team_two._win_rate >= rng_two):      
              team_two.win()              
              status = "WIN"
              if(team_one._name  == "Brasil" or team_two._name == "Brasil"):              
                match_history.append(team_one._name+" X "+ team_two._name)
              return team_two
            else:      
              status = "TIE" 
   
            
def initialize_list(original_team_list):
    base_name = "group"
    original_team_list = [[],[],[],[],[],[],[],[]]
    for j in range (0,8):
        current_file = base_name + str(j)+ ".txt"
        f = open(current_file, "r",encoding="utf-8-sig")        
        for line in f.readlines():
            ele = line.split(";")
            if(ele[0] != '\n'):
                ele[3] = ele[3][:-1]         
                temp_team = Team(ele[0],int(ele[2]),int(ele[3]),int(ele[1]))
                original_team_list[j].append(temp_team)
    return original_team_list

def ordenate_list_groups(current_team_list):
    for group in current_team_list:
        group.sort(key=lambda x: x._points, reverse=True)  
    return current_team_list

def extract_winners_groups(current_team_list):
    winner_list = []
    for group in current_team_list:
        winner_list.append(group[0])
        winner_list.append(group[1])
    return winner_list

In [4]:
%%time

k = 10000
current_team_list = [[],[],[],[],[],[],[],[]]
times_brazil_s_p = 0 
times_brazil_s_p_f = 0
times_brazil_s_p_s = 0
second_phase_teams = []
second_phase_frames = []
brazil_group_frames = []
brazil_matches_frames = []
brazil_matches_sucess_frames = []
coup_winners = []
            
for i in range(0,k):   
  current_team_list = initialize_list(current_team_list)
  t = open("jogos_copa.txt", "r",encoding="utf-8-sig")
  team_one = Team("",0,0,0)
  team_two = Team("",0,0,0)          

  for game in t.readlines():  
    team_one = search_team_one(game,current_team_list,team_one)
    team_two = search_team_two(game,current_team_list,team_two)  
    simulate_game(team_one,team_two,current_team_list)
    
  current_team_list = ordenate_list_groups(current_team_list)
  second_phase_teams = extract_winners_groups(current_team_list)
    
  if(current_team_list[6][0]._name == "Brasil"):
    times_brazil_s_p = times_brazil_s_p + 1
    times_brazil_s_p_f = times_brazil_s_p_f + 1
  if(current_team_list[6][1]._name == "Brasil"):
    times_brazil_s_p = times_brazil_s_p + 1
    times_brazil_s_p_s = times_brazil_s_p_s + 1
    
  found = False
  current_group_string = ""
  for team in current_team_list[6]:
    current_group_string = current_group_string + team._name + ":" + str(team._points) +"; "
  for groupAndCounter in brazil_group_frames:     
    if(groupAndCounter[0] == current_group_string):
      found = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found):
    brazil_group_frames.append([current_group_string,1]) 
    
  found_win_list =  False
  current_win_list = ""
  for team in second_phase_teams:
    current_win_list = current_win_list + team._name + ";"
  for groupAndCounter in second_phase_frames:     
    if(groupAndCounter[0] == current_win_list):
      found_win_list = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found_win_list):
    second_phase_frames.append([current_win_list,1]) 
    
  brazil_matches = []  
  first_a_team = second_phase_teams[0]
  second_a_team = second_phase_teams[1]
  first_b_team = second_phase_teams[2]
  second_b_team = second_phase_teams[3]
  first_c_team = second_phase_teams[4]
  second_c_team = second_phase_teams[5]
  first_d_team = second_phase_teams[6]
  second_d_team = second_phase_teams[7]    
  first_e_team = second_phase_teams[8]
  second_e_team = second_phase_teams[9]
  first_f_team = second_phase_teams[10]
  second_f_team = second_phase_teams[11]
  first_g_team = second_phase_teams[12]
  second_g_team = second_phase_teams[13]
  first_h_team = second_phase_teams[14]
  second_h_team = second_phase_teams[15]   
     
  team_49 = simulate_game_elm(first_a_team,second_b_team,brazil_matches)     
  team_50 = simulate_game_elm(first_c_team,second_d_team,brazil_matches) 
  team_51 = simulate_game_elm(first_d_team,second_c_team,brazil_matches)   
  team_52 = simulate_game_elm(first_b_team,second_a_team,brazil_matches)    
  team_53 = simulate_game_elm(first_e_team,second_f_team,brazil_matches)  
  team_54 = simulate_game_elm(first_g_team,second_h_team,brazil_matches) 
  team_55 = simulate_game_elm(first_f_team,second_e_team,brazil_matches)   
  team_56 = simulate_game_elm(first_h_team,second_g_team,brazil_matches)   
    
  team_58 = simulate_game_elm(team_53,team_54,brazil_matches)  
  team_57 = simulate_game_elm(team_49,team_50,brazil_matches) 
  team_60 = simulate_game_elm(team_55,team_56,brazil_matches)   
  team_59 = simulate_game_elm(team_51,team_52,brazil_matches) 
    
  team_61 = simulate_game_elm(team_57,team_58,brazil_matches)   
  team_62 = simulate_game_elm(team_59,team_60,brazil_matches)
    
  team_63 = simulate_game_elm(team_61,team_62,brazil_matches) 
    
  found_matches = False
  current_match_hist = ""
  for match in brazil_matches:
    current_match_hist = current_match_hist + match + "; "  
  for histAndCounterAndCounter in brazil_matches_frames:     
     if(histAndCounterAndCounter[0] == current_match_hist):
         if(team_63._name == "Brasil"):
            found_matches = True
            histAndCounterAndCounter[1] = histAndCounterAndCounter[1] + 1
            histAndCounterAndCounter[2] = histAndCounterAndCounter[2] + 1
         else:
            found_matches = True
            histAndCounterAndCounter[1] = histAndCounterAndCounter[1] + 1
  if(not found_matches):
    if(team_63._name == "Brasil"):
        brazil_matches_frames.append([current_match_hist,1,1])
    else:
        brazil_matches_frames.append([current_match_hist,1,0])
     
  if(team_63._name == "Brasil"):
    found_matches_sucess = False
    current_match_sucess_hist = ""
    for match in brazil_matches:
        current_match_sucess_hist = current_match_sucess_hist + match + "; "  
    for histAndCounter in brazil_matches_sucess_frames:     
     if(histAndCounter[0] == current_match_sucess_hist):       
        found_matches_sucess = True
        histAndCounter[1] = histAndCounter[1] + 1
    if(not found_matches_sucess):    
        brazil_matches_sucess_frames.append([current_match_sucess_hist,1])

  found_winner = False
  for teamAndCounter in coup_winners:     
     if(teamAndCounter[0] == team_63._name):
         found_winner = True
         teamAndCounter[1] = teamAndCounter[1] + 1
  if(not found_winner):
     coup_winners.append([team_63._name,1])
 

coup_winners.sort(key=lambda x: x[1], reverse=True)   
for group in coup_winners:
    group[1] = (group[1] / k) * 100
  
brazil_matches_sucess_frames.sort(key=lambda x: x[1], reverse=True)   
for group in brazil_matches_sucess_frames:
    group[1] = (group[1] / k) * 100   
    
brazil_matches_frames.sort(key=lambda x: x[2], reverse=True)   
for group in brazil_matches_frames:
    group[2] = (group[2] / group[1]) * 100
    group[1] = (group[1] / k) * 100  
    
brazil_group_frames.sort(key=lambda x: x[1], reverse=True)   
for group in brazil_group_frames:
    group[1] = (group[1] / k) * 100

second_phase_frames.sort(key=lambda x: x[1], reverse=True)   
for group in second_phase_frames:
    group[1] = (group[1] / k) * 100    

dt = datetime.now()

file_string = "logs/porcentagens_" + str(dt)
with open(file_string, 'w', encoding="utf-8") as f:
    f.write("Porcentagem de vezes que o Brasil classificou para 2 fase: "+ str(times_brazil_s_p / k * 100)+"%.")
    f.write("\nPorcentagem de vezes que o Brasil classificou para 2 fase (primeiro): "+ str(times_brazil_s_p_f / k * 100)+"%.")
    f.write("\nPorcentagem de vezes que o Brasil classificou para 2 fase (segundo): "+ str(times_brazil_s_p_s / k * 100)+"%.\n")
f.close()

file_string = "logs/grupo_do_brasil_" + str(dt)
with open(file_string, 'w', encoding="utf-8") as f:
    f.write("Quadros e porcentagens do grupo do Brasil:\n")
    for frame in brazil_group_frames:
        f.write("-> "+frame[0]+" "+str(frame[1])+"%.\n")
f.close()

file_string = "logs/lista_segunda_fase_" + str(dt)
with open(file_string, 'w', encoding="utf-8") as f:
    f.write("Quadros e porcentagens da segunda fase:\n")
    for frame in second_phase_frames:
        f.write("-> "+frame[0]+" "+str(frame[1])+"%.\n")
f.close()

file_string = "logs/campeoes_" + str(dt)
with open(file_string, 'w', encoding="utf-8") as f:
    f.write("Campeões:\n")
    for frame in coup_winners:
        f.write("-> "+frame[0]+" "+str(frame[1])+"%.\n")  
f.close()

file_string = "logs/caminho_brasileiro_sucesso_" + str(dt)
with open(file_string, 'w', encoding="utf-8") as f:        
    brazil_matches_frames.sort(key=lambda x: x[2], reverse=True)   
    f.write("Quadros e porcentagens do caminho Brasileiro: [ORDENADO POR SUCESSO]\n")
    for frame in brazil_matches_frames:
        if(frame[0] == ""):
            f.write("-> Não classificou. "+str(frame[1])+"%\n")
        else:
            f.write("-> "+frame[0]+" "+str(frame[1])+"%.\n\tSucesso: "+str(frame[2])+"%.\n")
f.close()

file_string = "logs/caminho_brasileiro_ocorrencia_" + str(dt)
with open(file_string, 'w', encoding="utf-8") as f:  
    brazil_matches_frames.sort(key=lambda x: x[1], reverse=True)   
    f.write("Quadros e porcentagens do caminho Brasileiro: [ORDENADO POR OCORRÊNCIA]\n")
    for frame in brazil_matches_frames:
        if(frame[0] == ""):
            f.write("-> Não classificou. "+str(frame[1])+"%\n")
        else:
            f.write("-> "+frame[0]+" "+str(frame[1])+"%.\n\tSucesso: "+str(frame[2])+"%.\n")
f.close()

file_string = "logs/caminho_brasileiro_vitorioso_" + str(dt)
with open(file_string, 'w', encoding="utf-8") as f:  
    f.write("Quadros e porcentagens do caminho vitorioso Brasileiro: [ORDENADO POR OCORRÊNCIA]\n")
    for frame in brazil_matches_sucess_frames:  
        f.write("-> "+frame[0]+" "+str(frame[1])+"%.\n")

CPU times: user 6.22 s, sys: 540 ms, total: 6.76 s
Wall time: 6.81 s


In [5]:
print("Porcentagem de vezes que o Brasil classificou para 2 fase: "+ str(times_brazil_s_p / k * 100)+"%.")
print("Porcentagem de vezes que o Brasil classificou para 2 fase (primeiro): "+ str(times_brazil_s_p_f / k * 100)+"%.")
print("Porcentagem de vezes que o Brasil classificou para 2 fase (segundo): "+ str(times_brazil_s_p_s / k * 100)+"%.")
print("\nQuadros e porcentagens do grupo do Brasil:")
for frame in brazil_group_frames[:3]:
    print("-> "+frame[0]+" "+str(frame[1])+"%.")
print("\nQuadros e porcentagens da segunda fase:")
for frame in second_phase_frames[:3]:
    print("-> "+frame[0]+" "+str(frame[1])+"%.")
print("\nCampeões:")
for frame in coup_winners[:10]:
    print("-> "+frame[0]+" "+str(frame[1])+"%.")
print("\nQuadros e porcentagens do caminho Brasileiro: [ORDENADO POR SUCESSO]")
for frame in brazil_matches_frames:
    if(frame[0] == ""):
        print("-> Não classificou."+str(frame[1])+"%")
    else:
        print("-> "+frame[0]+" "+str(frame[1])+"%.\n\tSucesso: "+str(frame[2])+"%.")
        
brazil_matches_frames.sort(key=lambda x: x[1], reverse=True)   
print("\nQuadros e porcentagens do caminho Brasileiro: [ORDENADO POR OCORRÊNCIA]")
for frame in brazil_matches_frames:
    if(frame[0] == ""):
        print("-> Não classificou."+str(frame[1])+"%")
    else:
        print("-> "+frame[0]+" "+str(frame[1])+"%.\n\tSucesso: "+str(frame[2])+"%.")
        
print("\nQuadros e porcentagens do caminho vitorioso Brasileiro: [ORDENADO POR OCORRÊNCIA]")
for frame in brazil_matches_sucess_frames:  
    print("-> "+frame[0]+" "+str(frame[1])+"%.")

Porcentagem de vezes que o Brasil classificou para 2 fase: 87.0%.
Porcentagem de vezes que o Brasil classificou para 2 fase (primeiro): 60.0%.
Porcentagem de vezes que o Brasil classificou para 2 fase (segundo): 27.0%.

Quadros e porcentagens do grupo do Brasil:
-> Camarões:5;Brasil:3;Suíça:3;Sérvia:2; 3.0%.
-> Brasil:5;Suíça:5;Sérvia:3;Camarões:1; 3.0%.
-> Camarões:5;Brasil:4;Sérvia:3;Suíça:2; 3.0%.

Quadros e porcentagens da segunda fase:
-> Equador;Senegal;Inglaterra;Gales;Polônia;Argentina;Austrália;Dinamarca;Espanha;Costa Rica;Canadá;Bélgica;Brasil;Sérvia;Coreia do Sul;Gana; 1.0%.
-> Equador;Holanda;Inglaterra;Estados Unidos;Arábia Saudita;Argentina;Austrália;Tunísia;Costa Rica;Alemanha;Bélgica;Canadá;Suíça;Camarões;Portugal;Gana; 1.0%.
-> Catar;Equador;Irã;Inglaterra;Arábia Saudita;Argentina;Austrália;França;Alemanha;Espanha;Croácia;Canadá;Camarões;Brasil;Coreia do Sul;Uruguai; 1.0%.

Campeões:
-> Brasil 20.0%.
-> Espanha 9.0%.
-> Irã 7.000000000000001%.
-> Inglaterra 7.000000000